![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [84]:
# Start coding...
import pandas as pd
import numpy as np

# Read in csv
df = pd.read_csv("bank_marketing.csv")


In [85]:
# Start coding...
import pandas as pd
import numpy as np

# Read in csv
marketing = pd.read_csv("bank_marketing.csv")

# Split into the three tables
client = marketing[["client_id", "age", "job", "marital", "education", 
             "credit_default", "housing", "loan"]]
campaign = marketing[["client_id", "campaign", "month", "day", 
               "duration", "pdays", "previous", "poutcome", "y"]]
economics = marketing[["client_id", "emp_var_rate", "cons_price_idx", 
                "euribor3m", "nr_employed"]]

# Rename client_id in the client table
client.rename(columns={"client_id": "id"}, inplace=True)

# Rename duration, y, and campaign columns
campaign.rename(columns={"duration": "contact_duration", 
                         "y": "campaign_outcome", 
                         "campaign": "number_contacts",
                         "previous": "previous_campaign_contacts",
                         "poutcome": "previous_outcome"}, 
                         inplace=True)

# Rename euribor3m and nr_employed
economics.rename(columns={"euribor3m": "euribor_three_months", 
                          "nr_employed": "number_employed"}, 
                          inplace=True)

# Clean education column
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)

# Clean job column
client["job"] = client["job"].str.replace(".", "")

# Change campaign_outcome to binary values
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, 
                                                                 "no": 0})

# Convert poutcome to binary values
campaign["previous_outcome"] = campaign["previous_outcome"].replace("nonexistent", 
                                                                    np.NaN)
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, 
                                                                 "failure": 0})

# Add campaign_id column
campaign["campaign_id"] = 1

# Capitalize month and day columns
campaign["month"] = campaign["month"].str.capitalize()

# Add year column
campaign["year"] = "2022"

# Convert day to string
campaign["day"] = campaign["day"].astype(str)

# Add last_contact_date column
campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]

# Convert to datetime
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], 
                                               format="%Y-%b-%d")

# Drop unneccessary columns
campaign.drop(columns=["month", "day", "year"], inplace=True)

# Save tables to individual csv files
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

# Store and print database_design
client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""


In [86]:

# Extract the client data
client = df[["client_id", "age", "job", "marital", "education", "credit_default", "housing", "loan"]]
client = client.rename(columns={"client_id": "id"})  # Rename the "client_id" column to "id"

# Extract the campaign data
campaign = df[["client_id", "campaign", "month", "day", "duration", "pdays", "previous", "poutcome", "y"]]

# Extract the economics data
economics = df[["client_id", "emp_var_rate", "cons_price_idx", "euribor3m", "nr_employed"]]

# Print the first few rows of each DataFrame to verify
print(client.head())
print(campaign.head())
print(economics.head())


   id  age        job  marital    education credit_default housing loan
0   0   56  housemaid  married     basic.4y             no      no   no
1   1   57   services  married  high.school        unknown      no   no
2   2   37   services  married  high.school             no     yes   no
3   3   40     admin.  married     basic.6y             no      no   no
4   4   56   services  married  high.school             no      no  yes
   client_id  campaign month  day  duration  pdays  previous     poutcome   y
0          0         1   may   13       261    999         0  nonexistent  no
1          1         1   may   19       149    999         0  nonexistent  no
2          2         1   may   23       226    999         0  nonexistent  no
3          3         1   may   27       151    999         0  nonexistent  no
4          4         1   may    3       307    999         0  nonexistent  no
   client_id  emp_var_rate  cons_price_idx  euribor3m  nr_employed
0          0           1.1       

In [87]:
# Rename columns
client.rename(columns={"client_id": "id"}, inplace=True)
campaign.rename(columns={"duration": "contact_duration", "previous":"previous_campaign_contacts", "y": "campaign_outcome", "poutcome": "previous_outcome", "campaign": "number_contacts"}, inplace=True)
economics.rename(columns={"euribor3m": "euribor_three_months","nr_employed": "number_employed"}, inplace=True)
# Print the first few rows of each DataFrame to verify
print(client.head())
print(campaign.head())
print(economics.head())

   id  age        job  marital    education credit_default housing loan
0   0   56  housemaid  married     basic.4y             no      no   no
1   1   57   services  married  high.school        unknown      no   no
2   2   37   services  married  high.school             no     yes   no
3   3   40     admin.  married     basic.6y             no      no   no
4   4   56   services  married  high.school             no      no  yes
   client_id  number_contacts  ... previous_outcome  campaign_outcome
0          0                1  ...      nonexistent                no
1          1                1  ...      nonexistent                no
2          2                1  ...      nonexistent                no
3          3                1  ...      nonexistent                no
4          4                1  ...      nonexistent                no

[5 rows x 9 columns]
   client_id  emp_var_rate  ...  euribor_three_months  number_employed
0          0           1.1  ...                 4.857  

In [88]:
# Clean the education column
client["education"] = client["education"].str.replace('.','_').replace("unknown", np.NaN)

# Remove periods from the job column
client["job"] = client["job"].apply(lambda x: x.replace(".", ""))
# Convert success, failure, and nonexistent values to 1, 0, and NaN respectively
campaign["previous_outcome"] = campaign["previous_outcome"].apply(lambda x: 1 if x == "success" else (0 if x == "failure" else np.NaN))
campaign["campaign_outcome"] = campaign["campaign_outcome"].apply(lambda x: 1 if x == "yes" else 0)

# Add a column called campaign_id in campaign, where all rows have a value of 1.
campaign["campaign_id"] = 1

# Create a datetime column called last_contact_date, in the format of "year-month-day", where the year is 2022, and the month and day values are taken from the "month" and "day" columns.
campaign["last_contact_date"] = pd.to_datetime("2022-" + campaign["month"] + "-" + campaign["day"].astype(str),                                          format="%Y-%b-%d")
campaign.head()

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,may,13,261,999,0,NaN,0,1,2022-05-13
1,1,1,may,19,149,999,0,NaN,0,1,2022-05-19
2,2,1,may,23,226,999,0,NaN,0,1,2022-05-23
3,3,1,may,27,151,999,0,NaN,0,1,2022-05-27
4,4,1,may,3,307,999,0,NaN,0,1,2022-05-03


In [89]:
economics.columns

Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')

In [90]:

# Drop unneccessary columns
campaign.drop(columns=["month", "day"], inplace=True)

# Save tables to individual csv files
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

# Store and print database_design
client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx clientFLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""

In [91]:
-- Query a CSV file:
select * from client;

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,no,yes,no
41184,41184,46,blue-collar,married,professional_course,no,no,no
41185,41185,56,retired,married,university_degree,no,yes,no
41186,41186,44,technician,married,professional_course,no,no,no


In [92]:
-- Query a CSV file:
select * from campaign;

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,NaN,0,1,2022-05-13 00:00:00+00:00
1,1,1,149,999,0,NaN,0,1,2022-05-19 00:00:00+00:00
2,2,1,226,999,0,NaN,0,1,2022-05-23 00:00:00+00:00
3,3,1,151,999,0,NaN,0,1,2022-05-27 00:00:00+00:00
4,4,1,307,999,0,NaN,0,1,2022-05-03 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,NaN,1,1,2022-11-30 00:00:00+00:00
41184,41184,1,383,999,0,NaN,0,1,2022-11-06 00:00:00+00:00
41185,41185,2,189,999,0,NaN,0,1,2022-11-24 00:00:00+00:00
41186,41186,1,442,999,0,NaN,1,1,2022-11-17 00:00:00+00:00


In [93]:
-- Query a variable:
-- SELECT * FROM df
--
-- Query a CSV file:
select * from economics;


,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0
...,...,...,...,...,...
41183,41183,-1.1,94.767,1.028,4963.6
41184,41184,-1.1,94.767,1.028,4963.6
41185,41185,-1.1,94.767,1.028,4963.6
41186,41186,-1.1,94.767,1.028,4963.6


In [94]:
# # Start coding...
# import pandas as pd
# import numpy as np

# # Read in csv
# marketing = pd.read_csv("bank_marketing.csv")

# # Split into the three tables
# client = marketing[["client_id", "age", "job", "marital", "education", 
#              "credit_default", "housing", "loan"]]
# campaign = marketing[["client_id", "campaign", "month", "day", 
#                "duration", "pdays", "previous", "poutcome", "y"]]
# economics = marketing[["client_id", "emp_var_rate", "cons_price_idx", 
#                 "euribor3m", "nr_employed"]]

# # Rename client_id in the client table
# client.rename(columns={"client_id": "id"}, inplace=True)

# # Rename duration, y, and campaign columns
# campaign.rename(columns={"duration": "contact_duration", 
#                          "y": "campaign_outcome", 
#                          "campaign": "number_contacts",
#                          "previous": "previous_campaign_contacts",
#                          "poutcome": "previous_outcome"}, 
#                          inplace=True)

# # Rename euribor3m and nr_employed
# economics.rename(columns={"euribor3m": "euribor_three_months", 
#                           "nr_employed": "number_employed"}, 
#                           inplace=True)

# # Clean education column
# client["education"] = client["education"].str.replace(".", "_")
# client["education"] = client["education"].replace("unknown", np.NaN)

# # Clean job column
# client["job"] = client["job"].str.replace(".", "")

# # Change campaign_outcome to binary values
# campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, 
#                                                                  "no": 0})

# # Convert poutcome to binary values
# campaign["previous_outcome"] = campaign["previous_outcome"].replace("nonexistent", 
#                                                                     np.NaN)
# campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, 
#                                                                  "failure": 0})

# # Add campaign_id column
# campaign["campaign_id"] = 1

# # Capitalize month and day columns
# campaign["month"] = campaign["month"].str.capitalize()

# # Add year column
# campaign["year"] = "2022"

# # Convert day to string
# campaign["day"] = campaign["day"].astype(str)

# # Add last_contact_date column
# campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]

# # Convert to datetime
# campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], 
#                                                format="%Y-%b-%d")

# # Drop unneccessary columns
# campaign.drop(columns=["month", "day", "year"], inplace=True)

# # Save tables to individual csv files
# client.to_csv("client.csv", index=False)
# campaign.to_csv("campaign.csv", index=False)
# economics.to_csv("economics.csv", index=False)

# # Store and print database_design
# client_table = """CREATE TABLE client
# (
#     id SERIAL PRIMARY KEY,
#     age INTEGER,
#     job TEXT,
#     marital TEXT,
#     education TEXT,
#     credit_default BOOLEAN,
#     housing BOOLEAN,
#     loan BOOLEAN
# );
# \copy client from 'client.csv' DELIMITER ',' CSV HEADER
# """

# campaign_table = """CREATE TABLE campaign
# (
#     campaign_id SERIAL PRIMARY KEY,
#     client_id SERIAL references client (id),
#     number_contacts INTEGER,
#     contact_duration INTEGER,
#     pdays INTEGER,
#     previous_campaign_contacts INTEGER,
#     previous_outcome BOOLEAN,
#     campaign_outcome BOOLEAN,
#     last_contact_date DATE    
# );
# \copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
# """

# economics_table = """CREATE TABLE economics
# (
#     client_id SERIAL references client (id),
#     emp_var_rate FLOAT,
#     cons_price_idx FLOAT,
#     euribor_three_months FLOAT,
#     number_employed FLOAT
# );
# \copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
# """